# Predicting Sea Levels
### Evaluate sea level rise over the next few years based on time-series analysis and Monte-Carlo simulation.

In [2]:
# Set up notebook headers
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pmdarima.arima import auto_arima
from scipy.stats import boxcox
from scipy.special import inv_boxcox
from sklearn.metrics import mean_squared_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.ar_model import AutoReg, ar_select_order
from statsmodels.tsa.stattools import adfuller
import yfinance as yf

%matplotlib inline
%load_ext autoreload
%autoreload 2

#### Data Collection
Read in an Excel file containing 5 sheets which have been compiled from various sources to reflect monthly measurements between June 2000 and July 2024:
- Sea Levels: Variation in sea levels (mm) relative to June 2000. Sourced from [NASA](https://climate.nasa.gov/vital-signs/sea-level/?intent=121).
- CO2 Levels: Concentration of carbon dioxide in the atmosphere (ppm). Sourced from [NASA](https://climate.nasa.gov/vital-signs/carbon-dioxide/?intent=121).
- Temperature: Average surface temperature (deg C). Sourced from 
- Ocean Warming: Energy stored in the oceans (ZJ) relative to June 2000. Sourced from 
- Polar Mass: Variation of polar mass in Antarctica (billion tons) relative to June 2000. Sourced from 

In [ ]:
dataset = pd.read_excel('C:\\Users\\rohit\\OneDrive\\2024\\Sea Level Predictions\\Project_Final.xlsx', 
                        sheet_name = ['Sea Levels', 'CO2 Levels', 'Temperature', 'Ocean Warming', 'Polar Mass'])

#### Visualise Raw Data
View some records in each table and plot them to highlight any key trends and/or outliers.

In [7]:
data_file['Sheet5']

,Date,Antarctica Change in Mass (billion tons)
0,2000-06-01,-406.8928
1,2000-07-01,-408.9063
2,2000-08-01,-410.9198
3,2000-09-01,-412.9334
4,2000-10-01,-414.9469
...,...,...
272,2023-02-01,-2919.6934
273,2023-03-01,-2821.3634
274,2023-04-01,-2610.5334
275,2023-05-01,-2756.5934
